## Задание

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru 
и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

*Наименование вакансии

*Предлагаемую зарплату (отдельно мин. отдельно макс. и отдельно валюту)

*Ссылку на саму вакансию

*Сайт откуда собрана вакансия

По своему желанию можно добавить еще работодателя и расположение. 
Данная структура должна быть одинаковая для вакансий с обоих сайтов. 
Общий результат можно вывести с помощью dataFrame через pandas.

In [13]:
import requests
import json
from pprint import pprint
from bs4 import BeautifulSoup as bs
from lxml import html
import pandas as pd
import re

In [14]:
request = 'python'

In [27]:
page = 0
max_page = 0

In [28]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}


In [29]:
vacancy_link_data = []
vacancy_name_data = []
vacancy_minsalary_data = []
vacancy_maxsalary_data = []
vacancy_currancy_data = []
vacancy_salary_data = []

In [30]:
while max_page != 40:
    link = f'https://hh.ru/search/vacancy?area=1&st=searchVacancy&text={request}&page={page}'
    response = requests.get(link, headers=header).text
    soup = bs(response,'lxml')
    vacancy_block = soup.find_all('div',{'class':'vacancy-serp'})[0]
    vacancy_list = vacancy_block.find_all('div',{'class':'vacancy-serp-item'})
    page += 1
    max_page += 1
    for vacancy in range(len(vacancy_list)):
        name1 = vacancy_block.find_all('span',{'class':'g-user-content'})[vacancy]
        name2 = name1.findChild()
        vacancy_name = name2.getText()
        vacancy_name_data.append(vacancy_name)
        vacancy_data={}
        vacancy_link1 = vacancy_block.find_all('span',{'class':'g-user-content'})[vacancy]
        vacancy_link = vacancy_link1.find('a')['href']
        vacancy_link_data.append(vacancy_link)
        salary1 = vacancy_block.find_all('div',{'class':'vacancy-serp-item__sidebar'})[vacancy]
        salary2 = str(salary1.findChild(recursive=False))
        salary3 = re.sub(r'\<[^>]*\>', '', salary2)

        if salary3 == None:
            salary = 0
        else:
            salary = salary3
            
        max_salary = 0
        min_salary = 0
        currency_salary = "NaN"

        if len(salary) > 1:
            salary_split = salary.split(" ")

            if salary_split[0] == 'от':
                max_salary = None
                min_salary = int(re.sub('\D',"",salary_split[1]))
                currency_salary = salary_split[2].replace(".","")  

            elif salary_split[0] == 'до':
                min_salary = None
                max_salary = int(re.sub('\D',"",salary_split[1]))
                currency_salary = salary_split[2].replace(".","")              

            else:
                salary2split = salary_split[0].split("-")
                if len(salary2split) > 1:
                    min_salary = int(re.sub('\D',"",salary2split[0]))
                    max_salary = int(re.sub('\D',"",salary2split[1]))
                    currency_salary = salary_split[1].replace(".","")   

        vacancy_minsalary_data.append(min_salary)
        vacancy_maxsalary_data.append(max_salary)
        vacancy_currancy_data.append(currency_salary)            
            
            
        vacancy_salary_data.append(salary)    

In [19]:
len(vacancy_salary_data)
        

2000

In [20]:
len(vacancy_list)

50

In [31]:
data = {'Vacancy Name': vacancy_name_data, 'Link': vacancy_link_data, 'Min_salary': vacancy_minsalary_data, 'Max_salary': vacancy_maxsalary_data, 'Currancy': vacancy_currancy_data,'Source': 'hh'}
df = pd.DataFrame(data)

In [32]:
df.head()

,Vacancy Name,Link,Min_salary,Max_salary,Currancy,Source
0,Программист Python,https://hh.ru/vacancy/36680277?query=python,160000.0,200000.0,руб,hh
1,Разработчик Python (Django),https://hh.ru/vacancy/36101850?query=python,0.0,0.0,NaN,hh
2,Разработчик Python 2.7 (3.Х),https://hh.ru/vacancy/36421719?query=python,250000.0,NaN,руб,hh
3,Программист Python,https://hh.ru/vacancy/34608956?query=python,0.0,0.0,NaN,hh
4,Python Developer SberDevices,https://hh.ru/vacancy/36645407?query=python,0.0,0.0,NaN,hh


In [34]:
df.to_csv('hh_vacancies.csv', index=False)